# Sudoku solver developing notebook

In [1]:
using Revise

In [2]:
function parse_sudoku(s)
    arrC = filter(x->!isspace(x), collect(s))
    grid = map(arrC) do c
        if c in "0."
            ' '
        else
            c
        end
    end
    grid = permutedims(reshape(grid, (9,9)), [2 1])
end

parse_sudoku (generic function with 1 method)

In [3]:
grid = parse_sudoku("."^81)
grid[:,:]

9×9 Array{Char,2}:
 ' '  ' '  ' '  ' '  ' '  ' '  ' '  ' '  ' '
 ' '  ' '  ' '  ' '  ' '  ' '  ' '  ' '  ' '
 ' '  ' '  ' '  ' '  ' '  ' '  ' '  ' '  ' '
 ' '  ' '  ' '  ' '  ' '  ' '  ' '  ' '  ' '
 ' '  ' '  ' '  ' '  ' '  ' '  ' '  ' '  ' '
 ' '  ' '  ' '  ' '  ' '  ' '  ' '  ' '  ' '
 ' '  ' '  ' '  ' '  ' '  ' '  ' '  ' '  ' '
 ' '  ' '  ' '  ' '  ' '  ' '  ' '  ' '  ' '
 ' '  ' '  ' '  ' '  ' '  ' '  ' '  ' '  ' '

In [4]:
file = "test/sudoku-hardest.txt"
function read_all(file::String)
    games = read(file, String)
    split(games)
end

read_all (generic function with 1 method)

In [5]:
games = read_all(file)
grid = parse_sudoku(games[1])

9×9 Array{Char,2}:
 '8'  '5'  ' '  ' '  ' '  '2'  '4'  ' '  ' '
 '7'  '2'  ' '  ' '  ' '  ' '  ' '  ' '  '9'
 ' '  ' '  '4'  ' '  ' '  ' '  ' '  ' '  ' '
 ' '  ' '  ' '  '1'  ' '  '7'  ' '  ' '  '2'
 '3'  ' '  '5'  ' '  ' '  ' '  '9'  ' '  ' '
 ' '  '4'  ' '  ' '  ' '  ' '  ' '  ' '  ' '
 ' '  ' '  ' '  ' '  '8'  ' '  ' '  '7'  ' '
 ' '  '1'  '7'  ' '  ' '  ' '  ' '  ' '  ' '
 ' '  ' '  ' '  ' '  '3'  '6'  ' '  '4'  ' '

In [6]:
box_from_index(indexes) = (indexes .- 1) .÷ 3 .+ 1
box_from_index((7,3))

(3, 1)

In [7]:
function indexes_of_box(box)
    start = (box .- 1) .* 3 .+ 1
    CartesianIndex(start .- 1) .+ CartesianIndices((3,3))
end
indexes_of_box((3,2))

3×3 CartesianIndices{2,Tuple{UnitRange{Int64},UnitRange{Int64}}}:
 CartesianIndex(7, 4)  CartesianIndex(7, 5)  CartesianIndex(7, 6)
 CartesianIndex(8, 4)  CartesianIndex(8, 5)  CartesianIndex(8, 6)
 CartesianIndex(9, 4)  CartesianIndex(9, 5)  CartesianIndex(9, 6)

In [8]:
get_box_peers(index) = indexes_of_box(box_from_index(index))
get_box_peers((5,5))

3×3 CartesianIndices{2,Tuple{UnitRange{Int64},UnitRange{Int64}}}:
 CartesianIndex(4, 4)  CartesianIndex(4, 5)  CartesianIndex(4, 6)
 CartesianIndex(5, 4)  CartesianIndex(5, 5)  CartesianIndex(5, 6)
 CartesianIndex(6, 4)  CartesianIndex(6, 5)  CartesianIndex(6, 6)

In [9]:
x = get_box_peers((1,1))
grid[x]

3×3 Array{Char,2}:
 '8'  '5'  ' '
 '7'  '2'  ' '
 ' '  ' '  '4'

In [10]:
function get_unitlist(index)
    row = [CartesianIndex((index[1], i)) for i in [1:9;]]
    col = [CartesianIndex((i, index[2])) for i in [1:9;]]
    box = get_box_peers(index)[:]
    [row, col, box]
end
unit = get_unitlist((5,5))

3-element Array{Array{CartesianIndex{2},1},1}:
 [CartesianIndex(5, 1), CartesianIndex(5, 2), CartesianIndex(5, 3), CartesianIndex(5, 4), CartesianIndex(5, 5), CartesianIndex(5, 6), CartesianIndex(5, 7), CartesianIndex(5, 8), CartesianIndex(5, 9)]
 [CartesianIndex(1, 5), CartesianIndex(2, 5), CartesianIndex(3, 5), CartesianIndex(4, 5), CartesianIndex(5, 5), CartesianIndex(6, 5), CartesianIndex(7, 5), CartesianIndex(8, 5), CartesianIndex(9, 5)]
 [CartesianIndex(4, 4), CartesianIndex(5, 4), CartesianIndex(6, 4), CartesianIndex(4, 5), CartesianIndex(5, 5), CartesianIndex(6, 5), CartesianIndex(4, 6), CartesianIndex(5, 6), CartesianIndex(6, 6)]

In [11]:
function get_peers(index)
    [x for x in Set(vcat(get_unitlist(index)...)) if x != CartesianIndex(index)]
end
peers = get_peers((5,5))

20-element Array{CartesianIndex{2},1}:
 CartesianIndex(2, 5)
 CartesianIndex(3, 5)
 CartesianIndex(4, 5)
 CartesianIndex(6, 5)
 CartesianIndex(5, 6)
 CartesianIndex(1, 5)
 CartesianIndex(6, 4)
 CartesianIndex(5, 3)
 CartesianIndex(5, 1)
 CartesianIndex(9, 5)
 CartesianIndex(5, 4)
 CartesianIndex(7, 5)
 CartesianIndex(5, 2)
 CartesianIndex(8, 5)
 CartesianIndex(4, 4)
 CartesianIndex(5, 7)
 CartesianIndex(5, 8)
 CartesianIndex(6, 6)
 CartesianIndex(4, 6)
 CartesianIndex(5, 9)

In [12]:
squares = CartesianIndices((9,9))
peers = Dict([(x, get_peers(x.I)) for x in squares])

Dict{CartesianIndex{2},Array{CartesianIndex{2},1}} with 81 entries:
  CartesianIndex(8, 4) => CartesianIndex{2}[CartesianIndex(8, 3), CartesianInde…
  CartesianIndex(3, 6) => CartesianIndex{2}[CartesianIndex(3, 2), CartesianInde…
  CartesianIndex(5, 6) => CartesianIndex{2}[CartesianIndex(1, 6), CartesianInde…
  CartesianIndex(4, 1) => CartesianIndex{2}[CartesianIndex(7, 1), CartesianInde…
  CartesianIndex(9, 1) => CartesianIndex{2}[CartesianIndex(7, 1), CartesianInde…
  CartesianIndex(4, 2) => CartesianIndex{2}[CartesianIndex(4, 5), CartesianInde…
  CartesianIndex(2, 2) => CartesianIndex{2}[CartesianIndex(3, 2), CartesianInde…
  CartesianIndex(5, 7) => CartesianIndex{2}[CartesianIndex(6, 7), CartesianInde…
  CartesianIndex(7, 4) => CartesianIndex{2}[CartesianIndex(1, 4), CartesianInde…
  CartesianIndex(2, 5) => CartesianIndex{2}[CartesianIndex(3, 5), CartesianInde…
  CartesianIndex(8, 3) => CartesianIndex{2}[CartesianIndex(8, 4), CartesianInde…
  CartesianIndex(8, 6) => CartesianIndex{

In [13]:
grid[peers[CartesianIndex(1, 1)]]

20-element Array{Char,1}:
 ' '
 '2'
 ' '
 ' '
 ' '
 ' '
 ' '
 ' '
 '3'
 '4'
 '5'
 '4'
 ' '
 '2'
 ' '
 ' '
 ' '
 '7'
 ' '
 ' '

In [14]:
units = Dict([(x, get_unitlist(x.I)) for x in CartesianIndices((9,9))[:]])

Dict{CartesianIndex{2},Array{Array{CartesianIndex{2},1},1}} with 81 entries:
  CartesianIndex(8, 4) => Array{CartesianIndex{2},1}[[CartesianIndex(8, 1), Car…
  CartesianIndex(3, 6) => Array{CartesianIndex{2},1}[[CartesianIndex(3, 1), Car…
  CartesianIndex(5, 6) => Array{CartesianIndex{2},1}[[CartesianIndex(5, 1), Car…
  CartesianIndex(4, 1) => Array{CartesianIndex{2},1}[[CartesianIndex(4, 1), Car…
  CartesianIndex(9, 1) => Array{CartesianIndex{2},1}[[CartesianIndex(9, 1), Car…
  CartesianIndex(4, 2) => Array{CartesianIndex{2},1}[[CartesianIndex(4, 1), Car…
  CartesianIndex(2, 2) => Array{CartesianIndex{2},1}[[CartesianIndex(2, 1), Car…
  CartesianIndex(5, 7) => Array{CartesianIndex{2},1}[[CartesianIndex(5, 1), Car…
  CartesianIndex(7, 4) => Array{CartesianIndex{2},1}[[CartesianIndex(7, 1), Car…
  CartesianIndex(2, 5) => Array{CartesianIndex{2},1}[[CartesianIndex(2, 1), Car…
  CartesianIndex(8, 3) => Array{CartesianIndex{2},1}[[CartesianIndex(8, 1), Car…
  CartesianIndex(8, 6) => Array{

In [15]:
nums =  [c for c in "123456789"]
function possible_solutions(grid)
    grid = map(grid) do c
        if c == ' '
            nums
        else
            [c]
        end
    end           
end
possible_solutions(grid)

9×9 Array{Array{Char,1},2}:
 ['8']                                          …  ['1', '2', '3', '4', '5', '6', '7', '8', '9']
 ['7']                                             ['9']                                        
 ['1', '2', '3', '4', '5', '6', '7', '8', '9']     ['1', '2', '3', '4', '5', '6', '7', '8', '9']
 ['1', '2', '3', '4', '5', '6', '7', '8', '9']     ['2']                                        
 ['3']                                             ['1', '2', '3', '4', '5', '6', '7', '8', '9']
 ['1', '2', '3', '4', '5', '6', '7', '8', '9']  …  ['1', '2', '3', '4', '5', '6', '7', '8', '9']
 ['1', '2', '3', '4', '5', '6', '7', '8', '9']     ['1', '2', '3', '4', '5', '6', '7', '8', '9']
 ['1', '2', '3', '4', '5', '6', '7', '8', '9']     ['1', '2', '3', '4', '5', '6', '7', '8', '9']
 ['1', '2', '3', '4', '5', '6', '7', '8', '9']     ['1', '2', '3', '4', '5', '6', '7', '8', '9']

In [16]:
function constraint1step(grid)
    grid= copy(grid)
    for s in squares 
        values = grid[s]
        len = length(values)
        
        if len == 1 # if square has only a value, the peers remove it
            grid[peers[s]] = map(grid[peers[s]]) do set
                setdiff(set, values)
            end
        elseif len == 0 # if square has zero values is inconsistent
            return grid, false    
        end
            
        for u in units[s] # if unit has only one place for value assign it
            for c in nums
                cs = [s for s in u if c in grid[s]]
                n_places = length(cs)
                if n_places == 1
                    grid[cs[1]] = [c]
                elseif n_places == 0 # if zero inconsistent
                    grid, false
                end
            end
        end
    end
    grid, true
end

constraint1step (generic function with 1 method)

In [17]:
function constraint(grid)
    grid, valid = constraint1step(grid)
    if !valid || all(map(length, grid) .== 1)
        return grid, valid
    end

    next_grid, next_valid = constraint1step(grid)

    if all(grid .== next_grid) && next_valid # fixed-pont
        return grid, true
    end

    if !valid || all(map(length, grid) .== 1)
        return grid, valid
    end
    
    constraint(next_grid)
end

constraint (generic function with 1 method)

In [18]:
grid = parse_sudoku("003020600900305001001806400008102900700000008006708200002609500800203009005010300")
grid_all = possible_solutions(grid)
constraint(grid_all)[1]

9×9 Array{Array{Char,1},2}:
 ['4']  ['8']  ['3']  ['9']  ['2']  ['1']  ['6']  ['5']  ['7']
 ['9']  ['6']  ['7']  ['3']  ['4']  ['5']  ['8']  ['2']  ['1']
 ['2']  ['5']  ['1']  ['8']  ['7']  ['6']  ['4']  ['9']  ['3']
 ['5']  ['4']  ['8']  ['1']  ['3']  ['2']  ['9']  ['7']  ['6']
 ['7']  ['2']  ['9']  ['5']  ['6']  ['4']  ['1']  ['3']  ['8']
 ['1']  ['3']  ['6']  ['7']  ['9']  ['8']  ['2']  ['4']  ['5']
 ['3']  ['7']  ['2']  ['6']  ['8']  ['9']  ['5']  ['1']  ['4']
 ['8']  ['1']  ['4']  ['2']  ['5']  ['3']  ['7']  ['6']  ['9']
 ['6']  ['9']  ['5']  ['4']  ['1']  ['7']  ['3']  ['8']  ['2']

In [82]:
function search((grid, valid))
    if !valid || all(map(length, grid) .== 1)
        return grid, valid
    end
    
     #minimum remaining values
    _, s = minimum([(length(grid[s]), s) for s in squares if length(grid[s]) >1])
    
    res1step = map(grid[s]) do v
        tmp = copy(grid)
        tmp[s] = [v]
        constraint(tmp)
    end
    resNstep = search.(res1step)

    reduce(resNstep, init=(grid, false)) do res1, res2
        if res1[2]
            res1
        else
            res2
        end
    end
        
end

search (generic function with 1 method)

In [83]:
function solve(grid)
    grid = possible_solutions(copy(grid))
    grid, valid = constraint(grid)

    search((grid, valid))
end

solve (generic function with 1 method)

In [84]:
function check_sol((grid, valid))
    if valid
        getindex.(grid, 1)
    else
        grid, valid
    end
end

check_sol (generic function with 1 method)

In [88]:
solve(parse_sudoku(games[1]))[1]

9×9 Array{Array{Char,1},2}:
 ['8']  ['5']  ['9']  ['6']  ['1']  ['2']  ['4']  ['3']  ['7']
 ['7']  ['2']  ['3']  ['8']  ['5']  ['4']  ['1']  ['6']  ['9']
 ['1']  ['6']  ['4']  ['3']  ['7']  ['9']  ['5']  ['2']  ['8']
 ['9']  ['8']  ['6']  ['1']  ['4']  ['7']  ['3']  ['5']  ['2']
 ['3']  ['7']  ['5']  ['2']  ['6']  ['8']  ['9']  ['1']  ['4']
 ['2']  ['4']  ['1']  ['5']  ['9']  ['3']  ['7']  ['8']  ['6']
 ['4']  ['3']  ['2']  ['9']  ['8']  ['1']  ['6']  ['7']  ['5']
 ['6']  ['1']  ['7']  ['4']  ['2']  ['5']  ['8']  ['9']  ['3']
 ['5']  ['9']  ['8']  ['7']  ['3']  ['6']  ['2']  ['4']  ['1']